# Importing packages; setting qpu

importing and setting dwave-2000q machine

In [1]:
import numpy as np
import dimod
import minorminer
import minorminer.layout as mml
%matplotlib inline
import dwave.inspector
import matplotlib as mpl
from datetime import datetime
from pathlib import Path  
import pandas as pd

In [2]:
import os
from dwave.system.samplers import DWaveSampler
from dwave.cloud.exceptions import *

try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus'},token="DEV-5fa1054cd25ed5a94ead77ab54c60e280f772f28")
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera'},token="DEV-5fa1054cd25ed5a94ead77ab54c60e280f772f28")
    qpus = {'Advantage': qpu_advantage, 'DW-2000Q': qpu_2000q}
    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")

from dwave.system import DWaveSampler
# Use a D-Wave system as the sampler
sampler = DWaveSampler(solver=dict(topology__type='chimera'),token="DEV-5fa1054cd25ed5a94ead77ab54c60e280f772f28")
print("QPU {} was selected.".format(sampler.solver.name))

Connected to Advantage Advantage_system6.1 and 2000Q DW_2000Q_6.
QPU DW_2000Q_6 was selected.


# Unit cell

Setting the chains of the Kagome unit cell

In [3]:
#making a dictionary of sites and their corresponding 2chains. There are 4 2chains per unit cell
sites_2chains={}
sites_2chains.update({1:(1,5)})
sites_2chains.update({4:(10,15)})
sites_2chains.update({7:(130,135)})
sites_2chains.update({10:(137,141)})
print(sites_2chains)

#making a dictionary of sites and their corresponding 3chains. There are 8 3chains per unit cell
sites_3chains={}
sites_3chains.update({0:[(2,6),(6,14)]})
sites_3chains.update({2:[(7,3),(3,131)]})
sites_3chains.update({3:[(9,12),(12,20)]})
sites_3chains.update({5:[(13,8),(8,136)]})
sites_3chains.update({6:[(129,132),(132,140)]})
sites_3chains.update({8:[(133,128),(128,256)]})
sites_3chains.update({9:[(138,142),(142,150)]})
sites_3chains.update({11:[(143,139),(139,267)]})
print(sites_3chains)

#this is a dictionary of all sites in a unit cell and their corresponding chains
sites_allchains={}
sites_allchains.update(sites_2chains)
sites_allchains.update(sites_3chains)        
print("there are", len(sites_allchains), "chains (sites) per unit cell")

{1: (1, 5), 4: (10, 15), 7: (130, 135), 10: (137, 141)}
{0: [(2, 6), (6, 14)], 2: [(7, 3), (3, 131)], 3: [(9, 12), (12, 20)], 5: [(13, 8), (8, 136)], 6: [(129, 132), (132, 140)], 8: [(133, 128), (128, 256)], 9: [(138, 142), (142, 150)], 11: [(143, 139), (139, 267)]}
there are 12 chains (sites) per unit cell


Writing the unit cell as a matrix

This includes:
2chains denoted by 2
3chains denotes by 3
intra-unit cell couplings denoted by J

These Jij values are not necessarily the values that
will be used in the embedding, but rather a way to 
discern the 3 types of couplings being used

In [4]:
#chains and couplings within a 2x2 star unit cell

J=0.5
unitcellmatrix = np.matrix([
    [0,  1,2,3,5,6,7,8,9,10,12,13,14,15,20,128,129,130,131,132,133,135,136,137,138,139,140,141,142,143,150,256,267],
    [1,  0,0,0,2,0,J,0,0,0, 0, 0, 0, 0, 0, 0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [2,  0,0,0,J,3,J,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [3,  0,0,0,0,0,3,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [5,  2,J,0,0,0,0,0,0,0, 0, J, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [6,  0,3,0,0,0,0,0,0,0, 0, 0, 3, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [7,  J,J,3,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [8,  0,0,0,0,0,0,0,0,0, 0, 3, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [9,  0,0,0,0,0,0,0,0,0, 3, J, 0, J, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [10, 0,0,0,0,0,0,0,0,0, 0, J, 0, 2, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0],
    [12, 0,0,0,0,0,0,0,3,0, 0, 0, 0, 0, 3, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [13, 0,0,0,J,0,0,3,J,J, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [14, 0,0,0,0,3,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [15, 0,0,0,0,0,0,0,J,2, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [20, 0,0,0,0,0,0,0,0,0, 3, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [128,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0],
    [129,J,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  3,  J,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [130,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  J,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [131,0,0,3,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [132,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0],
    [133,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 3,  J,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [135,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  J,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0],
    [136,0,0,0,0,0,0,3,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  0,  0,  0,  0,  0,  0],
    [137,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  J,  0,  0,  0],
    [138,0,0,0,0,0,0,0,0,J, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  J,  3,  J,  0,  0,  0],
    [139,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  3],
    [140,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  3,  0,  0,  J,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [141,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  2,  J,  0,  0,  0,  0,  0,  0,  0,  0],
    [142,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  3,  0,  0],
    [143,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  J,  0,  J,  J,  3,  0,  0,  0,  0,  0,  0,  0],
    [150,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0],
    [256,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [267,0,0,0,0,0,0,0,0,0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0]
])

%store unitcellmatrix

Stored 'unitcellmatrix' (matrix)


In [5]:
#this is an array that contains the sites that each of the qubits on the first row of the unitcell matrix correspond to
#this is to be used for tiled matrices which have the same bond structure but have different qubit numbers
sites_qubits=np.array([1,0,2,1,0,2,5,3,4,3,5,0,4,3,8,6,7,2,6,8,7,5,10,9,11,6,10,9,11,9,8,11])

l=[]
for i in range(1,33):
    a=unitcellmatrix[0,i]
    l.append(a)

l=np.asarray(l)
l=l.astype(int)
#print(l)

sites_qubits=np.row_stack((sites_qubits,l))
print(sites_qubits)

#now we have an array where the second row is the list of qubits in the unit cell
#the first row tells you which site each of those qubits are associated with

[[  1   0   2   1   0   2   5   3   4   3   5   0   4   3   8   6   7   2
    6   8   7   5  10   9  11   6  10   9  11   9   8  11]
 [  1   2   3   5   6   7   8   9  10  12  13  14  15  20 128 129 130 131
  132 133 135 136 137 138 139 140 141 142 143 150 256 267]]


In [6]:
# getting a list of the qubits included in a unit cell. The first row in the big matrix is the list of qubits.
unitcellqubits=unitcellmatrix[0]
#conoverting to array
unitcellqubits=np.asarray(unitcellqubits)
#converting from floats to ints
unitcellqubits=unitcellqubits.astype(int)
unitcellqubits=unitcellqubits[0]
#deleting the 0 since it was a placeholder in the big matrix
unitcellqubits=np.delete(unitcellqubits,0)
print(unitcellqubits)
print("There are", len(unitcellqubits), "qubits per unit cell")

[  1   2   3   5   6   7   8   9  10  12  13  14  15  20 128 129 130 131
 132 133 135 136 137 138 139 140 141 142 143 150 256 267]
There are 32 qubits per unit cell


Writing a matrix for tiling the unit cell
There are 
3 +x tiling couplings
3 -y tiling couplings
1 xy coupling
per unit cell (different for unit cells on an edge,
where there isn't a nearby unit cell to tile to

In [7]:
#the +x coupling
tilingx = np.matrix([
    [0,  15,23,141,149,150,147],
    [15, 0, J, 0,  0,  0,  0],
    [23, J, 0, 0,  0,  0,  0],
    [141,0, 0, 0,  J,  0,  0],
    [149,0, 0, J,  0,  0,  0],
    [150,0, 0, 0,  0,  0,  J],
    [147,0, 0, 0,  0,  J,  0]
])
#the -y coupling
tilingy = np.matrix([
    [0,  130,258,137,265,267,270],
    [130,0,  J,  0,  0,  0,  0],
    [258,J,  0,  0,  0,  0,  0],
    [137,0,  0,  0,  J,  0,  0],
    [265,0,  0,  J,  0,  0,  0],
    [267,0,  0,  0,  0,  0,  J],
    [270,0,  0,  0,  0,  J,  0]
])
#the special -x-y coupling
xymatrix = np.matrix([
    [0,  272,276],
    [272,0,  J],
    [276,J,  0]
])

Getting all of the couplers in the unit cell as a dictionary

In [8]:
#keeping a record of the 2 qubit chains and their coupling values
couplings_2chains={}
jvalue_2chains=-2.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==2:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (1,5) and (5,1)
            list.sort()
            couplings_2chains.update({(list[0],list[1]):jvalue_2chains})
print(couplings_2chains)
print("There are",len(couplings_2chains),"2chains per unit cell")

#keeping a record of the 3 qubit chains and their coupling values
couplings_3chains={}
jvalue_3chains=-3.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==3:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_3chains.update({(list[0],list[1]):jvalue_3chains})
print(couplings_3chains)
print("There are",len(couplings_3chains),"3chains per unit cell (when split into 2chains)")

#keeping a record of the internal couplings and their coupling values
couplings_internal={}
jvalue_internal=-1.0
for i in range(1,unitcellmatrix.shape[0]):
    for j in range(1,unitcellmatrix.shape[0]):
        if unitcellmatrix[i,j]==J:
            index1=i
            index2=j
            qubit1=unitcellmatrix[i,0]
            qubit2=unitcellmatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_internal.update({(list[0],list[1]):jvalue_internal})
print(couplings_internal)
print("There are",len(couplings_internal),"internal couplings per unit cell")

#keeping a record of the tiling couplings and their coupling values
couplings_tilingx={}
jvalue_tiling=-1.0
for i in range(1,tilingx.shape[0]):
    for j in range(1,tilingx.shape[0]):
        if tilingx[i,j]==J:
            index1=i
            index2=j
            qubit1=tilingx[i,0]
            qubit2=tilingx[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_tilingx.update({(list[0],list[1]):jvalue_tiling})
print(couplings_tilingx)
print("There are",len(couplings_tilingx),"+x tiling couplings")

couplings_tilingy={}
jvalue_tiling=-1.0
for i in range(1,tilingy.shape[0]):
    for j in range(1,tilingy.shape[0]):
        if tilingy[i,j]==J:
            index1=i
            index2=j
            qubit1=tilingy[i,0]
            qubit2=tilingy[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_tilingy.update({(list[0],list[1]):jvalue_tiling})
print(couplings_tilingy)
print("There are",len(couplings_tilingy),"-y tiling couplings")

couplings_xy={}
jvalue_tiling=-1.0
for i in range(1,xymatrix.shape[0]):
    for j in range(1,xymatrix.shape[0]):
        if xymatrix[i,j]==J:
            index1=i
            index2=j
            qubit1=xymatrix[i,0]
            qubit2=xymatrix[0,j]
            list=[qubit1,qubit2]
            #this gets rid of repeats. ie, (2,6) and (6,2)
            list.sort()
            couplings_xy.update({(list[0],list[1]):jvalue_tiling})
print(couplings_xy)
print("There are",len(couplings_xy),"xy couplings")

#Dictionary with all chains, internal couplings, and tiling couplings
couplings_all_unit={}
couplings_all_unit.update(couplings_2chains)
couplings_all_unit.update(couplings_3chains)
couplings_all_unit.update(couplings_internal)
couplings_all_unit.update(couplings_tilingx)
couplings_all_unit.update(couplings_tilingy)
couplings_all_unit.update(couplings_xy)

print("There are",len(couplings_all_unit),"total couplings per unit cell")

{(1.0, 5.0): -2.0, (10.0, 15.0): -2.0, (130.0, 135.0): -2.0, (137.0, 141.0): -2.0}
There are 4 2chains per unit cell
{(2.0, 6.0): -3.0, (3.0, 7.0): -3.0, (3.0, 131.0): -3.0, (6.0, 14.0): -3.0, (8.0, 13.0): -3.0, (8.0, 136.0): -3.0, (9.0, 12.0): -3.0, (12.0, 20.0): -3.0, (128.0, 133.0): -3.0, (128.0, 256.0): -3.0, (129.0, 132.0): -3.0, (132.0, 140.0): -3.0, (138.0, 142.0): -3.0, (139.0, 143.0): -3.0, (139.0, 267.0): -3.0, (142.0, 150.0): -3.0}
There are 16 3chains per unit cell (when split into 2chains)
{(1.0, 7.0): -1.0, (1.0, 129.0): -1.0, (2.0, 5.0): -1.0, (2.0, 7.0): -1.0, (5.0, 13.0): -1.0, (9.0, 13.0): -1.0, (9.0, 15.0): -1.0, (10.0, 13.0): -1.0, (10.0, 138.0): -1.0, (129.0, 133.0): -1.0, (129.0, 135.0): -1.0, (130.0, 133.0): -1.0, (135.0, 143.0): -1.0, (136.0, 140.0): -1.0, (137.0, 143.0): -1.0, (138.0, 141.0): -1.0, (138.0, 143.0): -1.0}
There are 17 internal couplings per unit cell
{(15.0, 23.0): -1.0, (141.0, 149.0): -1.0, (147.0, 150.0): -1.0}
There are 3 +x tiling couplings


Checking all of the couplers in the unit cell

In [9]:
#checking if all of the couplers in the unit cell are present in the qpu
import termcolor
from termcolor import colored

if all(coupler in sampler.edgelist for coupler in couplings_all_unit.keys())==True:
    print("all couplers in unit cell are good")
else:
    for coupler in couplings_all_unit.keys():
        if coupler in sampler.edgelist:
            print(colored(coupler, 'green'), colored('is found in list', 'green'))
        else:
            print(colored(coupler, 'red'), colored('is not found in list', 'red'))

all couplers in unit cell are good


Defining matrices for tiling the 1+3 matrices.
For each matrix, there is a matrix for tiling right and one for tiling down
(8 different tiling matrices)

In [10]:
#this matrix will be added on to the base unit cell matrix to tile to the right (+16)
unitcell_plus16 = np.zeros(unitcellmatrix.shape)
for i in range(1,unitcellmatrix.shape[0]):
    unitcell_plus16[0][i]=16
    unitcell_plus16[i][0]=16
unitcell_plus16=np.asmatrix(unitcell_plus16)

#this matrix will be added on to the base unit cell matrix to tile down (+256)
unitcell_plus256 = np.zeros(unitcellmatrix.shape)
for i in range(1,unitcellmatrix.shape[0]):
    unitcell_plus256[0][i]=256
    unitcell_plus256[i][0]=256
unitcell_plus256=np.asmatrix(unitcell_plus256)

#this matrix will be added on to the base tilingx matrix to tile to the right (+16)
tilingx_plus16 = np.zeros(tilingx.shape)
for i in range(1,tilingx.shape[0]):
    tilingx_plus16[0][i]=16
    tilingx_plus16[i][0]=16
tilingx_plus16=np.asmatrix(tilingx_plus16)

#this matrix will be added on to the base tilingx matrix to tile down (+256)
tilingx_plus256 = np.zeros(tilingx.shape)
for i in range(1,tilingx.shape[0]):
    tilingx_plus256[0][i]=256
    tilingx_plus256[i][0]=256
tilingx_plus256=np.asmatrix(tilingx_plus256)

#this matrix will be added on to the base tilingy matrix to tile to the right (+16)
tilingy_plus16 = np.zeros(tilingy.shape)
for i in range(1,tilingy.shape[0]):
    tilingy_plus16[0][i]=16
    tilingy_plus16[i][0]=16
tilingy_plus16=np.asmatrix(tilingy_plus16)

#this matrix will be added on to the base tilingy matrix to tile down (+256)
tilingy_plus256 = np.zeros(tilingy.shape)
for i in range(1,tilingy.shape[0]):
    tilingy_plus256[0][i]=256
    tilingy_plus256[i][0]=256
tilingy_plus256=np.asmatrix(tilingy_plus256)

#this matrix will be added on to the base xy matrix to tile to the right (+16)
xy_plus16 = np.zeros((3,3))
for i in range(1,xymatrix.shape[0]):
    xy_plus16[0][i]=16
    xy_plus16[i][0]=16
xy_plus16=np.asmatrix(xy_plus16)

#this matrix will be added on to the base xy matrix to tile down (+256)
xy_plus256 = np.zeros((3,3))
for i in range(1,xymatrix.shape[0]):
    xy_plus256[0][i]=256
    xy_plus256[i][0]=256
xy_plus256=np.asmatrix(xy_plus256)

# Functions for tiling

Defining a function that tiles any matrix to the right,
and one that tiles any matrix down.
The functions add the appropriate tiling matrix
The output is a tuple of matrices

In [11]:
#function that tiles a unitcell to the right once
def tile_right(unitcellmat,tilingmatx,tilingmaty,xymat):
    
    tileright_unitcell=unitcellmat+unitcell_plus16
    tileright_tilingx=tilingmatx+tilingx_plus16
    tileright_tilingy=tilingmaty+tilingy_plus16
    tileright_xy=xymat+xy_plus16
    
    return tileright_unitcell,tileright_tilingx,tileright_tilingy,tileright_xy


#function that tiles a unitcell down once
def tile_down(unitcellmat,tilingmatx,tilingmaty,xymat):
    
    tiledown_unitcell=unitcellmat+unitcell_plus256
    tiledown_tilingx=tilingmatx+tilingx_plus256
    tiledown_tilingy=tilingmaty+tilingy_plus256
    tiledown_xy=xymat+xy_plus256
    
    return tiledown_unitcell,tiledown_tilingx,tiledown_tilingy,tiledown_xy

defining a function that tiles the unit cell to make a NxN lattice.
Cuts off dangling tiling couplers at the boundaries
Outputs a tuple of:
unitcellregistry - list of unit cell matrices
tiling(x/y/xy)registry - list of tiling matrices

In [12]:
#NxN tiling: This function tiles in +x direction N-1 times and -y direction N-1 times. 
#Output is basically a NxN lattice
def NxN_tile(N):
    #making a registry of unitcells. The first entry is the base unitcell
    unitcellregistry=[]
    unitcellregistry.append(unitcellmatrix)
    #making a registry of x tilings. The first entry is the base tiling
    tilingxregistry=[]
    tilingxregistry.append(tilingx)
    #making a registry of y tilings. The first entry is the base tiling
    tilingyregistry=[]
    tilingyregistry.append(tilingy)
    #making a registry of xy tilings. The first entry is the base tiling
    xyregistry=[]
    xyregistry.append(xymatrix)
    
    #notes about tiling.
    #unitcell tiles right N-1 times, tiles down N-1 times
    #x tiling tiles right N-2 times, y tiling tiles right N-1 times, xy tiling tiles right N-2 times
    #x tiling tiles down N-1 times, y tiling tiles down N-2 times, xy tiling tiles down N-2 times
    
    
    #tiling the unit cell to the right N-1 times, and appending the registries with the new tiled matrices
    for i in range(N-1):
        newmatsright=tile_right(unitcellregistry[i],tilingxregistry[0],tilingyregistry[0],xyregistry[0])
        unitcellregistry.append(newmatsright[0])
    #tiling the tilingx matrix to the right N-2 times (to cut off the qubits hanging) , and appending the registries with the new tiled matrices
    for i in range(N-2):
        newmatsright=tile_right(unitcellregistry[0],tilingxregistry[i],tilingyregistry[0],xyregistry[0])
        tilingxregistry.append(newmatsright[1])
    #tiling the tilingy matrix to the right N-1 times (to cut off the qubits hanging) , and appending the registries with the new tiled matrices
    for i in range(N-1):
        newmatsright=tile_right(unitcellregistry[0],tilingxregistry[0],tilingyregistry[i],xyregistry[0])
        tilingyregistry.append(newmatsright[2])
   #tiling the xy matrix to the right N-2 times (to cut off the qubits hanging) , and appending the registries with the new tiled matrices
    for i in range(N-2):
        newmatsright=tile_right(unitcellregistry[0],tilingxregistry[0],tilingyregistry[0],xyregistry[i])
        xyregistry.append(newmatsright[3])
    
    #Now there is a row of N unit cells in the registries. For each unit cell in this row, we tile down N-1 times.
    #a list containing the row to be tiled down. Initialize with 1st row. It will be updated with each iteration
    tobetiled_downuc=[]
    tobetiled_downx=[]
    tobetiled_downy=[]
    tobetiled_downxy=[]
    
    for i in range(len(unitcellregistry)):
        tobetiled_downuc.append(unitcellregistry[i])   
    for i in range(len(tilingxregistry)):
        tobetiled_downx.append(tilingxregistry[i])
    for i in range(len(tilingyregistry)):
        tobetiled_downy.append(tilingyregistry[i])
    for i in range(len(xyregistry)):
        tobetiled_downxy.append(xyregistry[i])
    
    
    for i in range(N-1):
        newrow=[]
        for j in tobetiled_downuc:
            #tile down each unit cell in the preceding row
            newmatsdownuc=tile_down(j,tilingxregistry[0],tilingyregistry[0],xyregistry[0])
            #update the registry with new matrix
            unitcellregistry.append(newmatsdownuc[0])
            #populate newrow with the new matrices
            newrow.append(newmatsdownuc[0])
            #update the to be tiled down list to be used in the new iteration
            tobetiled_downuc=newrow
            
    for i in range(N-1):
        newrow=[]
        for j in tobetiled_downx:
            #tile down each tilingx matrix in the preceding row
            newmatsdownx=tile_down(unitcellregistry[0],j,tilingyregistry[0],xyregistry[0])
            #update the registry with new matrix
            tilingxregistry.append(newmatsdownx[1])
            #populate newrow with the new matrices
            newrow.append(newmatsdownx[1])
            #update the to be tiled down list to be used in the new iteration
            tobetiled_downx=newrow
    
    for i in range(N-2):
        newrow=[]
        for j in tobetiled_downy:
            #tile down each tilingy matrix in the preceding row
            newmatsdowny=tile_down(unitcellregistry[0],tilingxregistry[0],j,xyregistry[0])
            #update the registry with new matrix
            tilingyregistry.append(newmatsdowny[2])
            #populate newrow with the new matrices
            newrow.append(newmatsdowny[2])
            #update the to be tiled down list to be used in the new iteration
            tobetiled_downy=newrow
    
    for i in range(N-2):
        newrow=[]
        for j in tobetiled_downxy:
            #tile down each xy matrix in the preceding row
            newmatsdownxy=tile_down(unitcellregistry[0],tilingxregistry[0],tilingyregistry[0],j)
            #update the registry with new matrix
            xyregistry.append(newmatsdownxy[3])
            #populate newrow with the new matrices
            newrow.append(newmatsdownxy[3])
            #update the to be tiled down list to be used in the new iteration
            tobetiled_downxy=newrow
            
    if N==1:
        tilingxregistry=[]
        tilingyregistry=[]
        xyregistry=[]
    
    return unitcellregistry,tilingxregistry,tilingyregistry,xyregistry

Using sites_qubits from earlier, we create an array that contains arrays of the form of sites_qubits, but for the whole NxN lattice

In [13]:
def tile_sites_qubits(N):
    all_sites_qubits=[]
    all_sites_qubits.append(sites_qubits)
    
    #creating the first row
    for i in range(N-1):
        row1=all_sites_qubits[i][0]
        new_row2=all_sites_qubits[i][1]+16
        new_array_right=np.row_stack((row1,new_row2))
        all_sites_qubits.append(new_array_right)
    
    tobetiled_down=[]
    for i in range(N):
        tobetiled_down.append(all_sites_qubits[i])
    
    
    for i in range(N-1):
        
        newrow=[]
        for j in tobetiled_down:
            #tile down each unit cell in the preceding row
            row1=j[0]
            new_row2=j[1]+256
            new_array_down=np.row_stack((row1,new_row2))
            
            #update the registry with each new matrix
            all_sites_qubits.append(new_array_down)
        
            #populate newrow with the new matrices
            newrow.append(new_array_down)
            #update the to be tiled down list to be used in the new iteration
            tobetiled_down=newrow
    
    all_sites_qubits=np.asarray(all_sites_qubits)
    all_sites_qubits=np.reshape(all_sites_qubits,(N,N,2,32))
    
    return all_sites_qubits

dwave machine only likes dictionaries for couplers.
defining a function to wrrite all of the couplers as a dictionary

In [14]:
#a function for defining all of the coupling dictionaries of the form (qubit1,qubit2):jvalue
def getcouplings(ucreg,tilregx,tilregy,xyreg,
                 jvalue_2chains,jvalue_3chains,jvalue_internal,jvalue_tiling):
    
    couplings_2chains={}    
    couplings_3chains={}   
    couplings_internal={}
    couplings_tilingx={}
    couplings_tilingy={}
    couplings_xy={}
    
#     index_sites = []
#     avoid_third_site = []
    
    
    
    for matrix in ucreg:
        #keeping a record of the 2 qubit chains and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==2:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (1,5) and (5,1)
                    list.sort()
                    couplings_2chains.update({(list[0],list[1]):jvalue_2chains})
#                     index_sites.append(list[0])
        #keeping a record of the 3 qubit chains and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==3:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_3chains.update({(list[0],list[1]):jvalue_3chains})
#                     if list[0] not in avoid_third_site:
#                         index_sites.append(list[0])
#                     avoid_third_site.append(list[1], list[0])
        #keeping a record of the internal couplings and their coupling values
        for i in range(1,unitcellmatrix.shape[0]):
            for j in range(1,unitcellmatrix.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_internal.update({(list[0],list[1]):jvalue_internal})
        
        
        
    for matrix in tilregx:
        #keeping a record of the x tiling couplings and their coupling values
        for i in range(1,tilingx.shape[0]):
            for j in range(1,tilingx.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_tilingx.update({(list[0],list[1]):jvalue_tiling})
                    
                    
    for matrix in tilregy:
        #keeping a record of the y tiling couplings and their coupling values
        for i in range(1,tilingy.shape[0]):
            for j in range(1,tilingy.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_tilingy.update({(list[0],list[1]):jvalue_tiling})
                    
                    
    
    for matrix in xyreg:
        #keeping a record of the xy couplings and their coupling values
        for i in range(1,xymatrix.shape[0]):
            for j in range(1,xymatrix.shape[0]):
                if matrix[i,j]==J:
                    index1=i
                    index2=j
                    qubit1=matrix[i,0]
                    qubit2=matrix[0,j]
                    list=[qubit1,qubit2]
                    #this gets rid of repeats. ie, (2,6) and (6,2)
                    list.sort()
                    couplings_xy.update({(list[0],list[1]):jvalue_tiling})
                    

    #Dictionary with all chains, internal couplings, and tiling couplings
    all_couple_values={}
    all_couple_values.update(couplings_2chains)
    all_couple_values.update(couplings_3chains)
    all_couple_values.update(couplings_internal)
    all_couple_values.update(couplings_tilingx)
    all_couple_values.update(couplings_tilingy)
    all_couple_values.update(couplings_xy)
    
    
    #print(couplings_2chains)
    print("There are",len(couplings_2chains),"2chains")
    
    #print(couplings_3chains)
    print("There are",len(couplings_3chains),"3chains (when split into 2chains)")
            
    #print(couplings_internal)
    print("There are",len(couplings_internal),"internal couplings")
    
    #print(couplings_tilingx)
    print("There are",len(couplings_tilingx),"x tiling couplings")
    
    #print(couplings_tilingy)
    print("There are",len(couplings_tilingy),"y tiling couplings")
    
    #print(couplings_xy)
    print("There are",len(couplings_xy),"xy couplings")
    
    #print(all_couple_values)
    print("There are",len(all_couple_values),"total couplings")
    
    return all_couple_values

# Setting N and J, running tiling functions

Running tiling functions

In [15]:
#setting N and J values
Num=7
j2=-2.0
j3=-3.0
jint=-1
jtil=-1

In [16]:
registries=NxN_tile(Num)
all_sites_qubits=tile_sites_qubits(Num)

In [17]:
couplers=getcouplings(
    registries[0],registries[1],registries[2],registries[3],
j2,j3,jint,jtil)

There are 196 2chains
There are 784 3chains (when split into 2chains)
There are 833 internal couplings
There are 126 x tiling couplings
There are 126 y tiling couplings
There are 36 xy couplings
There are 2101 total couplings


# Functions for: checking all qubits, deleting sites with missing qubits

In [18]:
#checking nodes and making a list of missing ones
def check_nodes(nodlist):
    missingqs=[]
    if all(qubit in sampler.nodelist for qubit in nodlist)==True:
        print("all nodes are good")
    else:
        for node in nodlist:
            if node not in sampler.nodelist:
                missingqs.append(node)
                print(colored(node, 'red'), colored('is not found in list', 'red'))
    return missingqs

In [19]:
def split_oldcouplers(oldcoups,oldtups):    
    oldinttillist=[]
    old2chainslist=[]
    old3chainslistpairs=[]

    for i in range(len(oldtups)):
        tupl=oldtups[i]
        if (oldcoups[tupl]==jint):
            oldinttillist.append(tupl)
        if (oldcoups[tupl]==jtil):
            oldinttillist.append(tupl)
        if oldcoups[tupl]==j2:
            old2chainslist.append(tupl)
        if oldcoups[tupl]==j3:
            old3chainslistpairs.append(tupl)

    #writing 3chains as triplets, and keeping a record of the two pairs that each triplet corresponds to
    iset = set([frozenset(s) for s in old3chainslistpairs])  # Convert to a set of sets
    result = []
    result_pair = []
    while(iset):                  # While there are sets left to process:
        nset = set(iset.pop())      # Pop a new set
        nset_pair = tuple(nset)
        check = len(iset)           # Does iset contain more sets
        while check:                # Until no more sets to check:
            check = False
            for s in iset.copy():       # For each other set:
                if nset.intersection(s):  # if they intersect:
                    check = True            # Must recheck previous sets
                    result_pair.append([nset_pair, tuple(s)])
                    iset.remove(s)          # Remove it from remaining sets
                    nset.update(s)          # Add it to the current set
        result.append(tuple(nset))  # Convert back to a list of tuples
    old3chainslist=result

    return oldinttillist,old2chainslist,old3chainslist,result_pair

In [20]:
def to_remove(missingqs,oldinttillist,old2chainslist,old3chainslist,result_pair):
    #make lists of all 3chains, 2chains, and inttils to remove
    remove3chains=[]
    remove3chains_pairs = []
    remove2chains=[]
    removeinttil=[]
    for i in range(len(missingqs)):
        missq = missingqs[i]

        #for each missq, check if it's in a 2chain or 3chain
        for i, tup3chain in enumerate(old3chainslist):
            if (tup3chain[0] == missq or tup3chain[1] == missq
                    or tup3chain[2] == missq):
                chain=tup3chain
                remove3chains.append(chain)

        for i, tup2chain in enumerate(old2chainslist):
            if (tup2chain[0] == missq or tup2chain[1] == missq):
                chain=tup2chain
                remove2chains.append(chain)

        #identify all internal/tiling couplings assoociated with the chain
        for qubit in chain:
            for i, tupinttil in enumerate(oldinttillist):
                if (tupinttil[0] == qubit or tupinttil[1] == qubit):
                    removeinttil.append(tupinttil)

    #remove repeats in removeinttil
    removeinttilnew=[]
    removeinttil=set(removeinttil)
    for tup in removeinttil:
        removeinttilnew.append(tup)
    removeinttil=removeinttilnew


    #remove3chains is a list of triplets. have to convert into pairs
    for i in range(len(remove3chains)):
        triplet=remove3chains[i]
        for j in range(len(old3chainslist)):
            if old3chainslist[j]==triplet:
                pairs=result_pair[j]
                for pair in pairs:
                    remove3chains_pairs.append(pair)

    #combining the remove lists
    allremove=[]
    for tup in remove3chains_pairs:
        allremove.append(tup)
    for tup in remove2chains:
        allremove.append(tup)
    for tup in removeinttil:
        allremove.append(tup)
    
    return allremove

In [21]:
def Reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup

In [22]:
def remove_couplers(couplerlist,oldtups,allremove):
    #removing the couplers from the master couplers list
    print(len(couplerlist), "couplers before")
    for tup in allremove:
        if tup in oldtups:
            print(tup, "is good, removing from couplers")
            couplerlist.pop(tup)
        else:
            print(tup,"is bad")
            if Reverse(tup) in oldtups:
                print(Reverse(tup),"is good, removing from couplers")
                couplerlist.pop(Reverse(tup))
            else:
                print("what the")

    #extracting tuples from coupler dictionary
    newtuples_only=[]
    for edge in couplerlist.keys():
        newtuples_only.append(edge)

    print(len(couplerlist), "couplers after")
    
    #recheck tuples
    if all(edge in sampler.edgelist for edge in newtuples_only)==True:
        print("all couplers are good")
    
    return couplerlist

# Old flux calibration functions

This function first sets all Ji and hi to zero for the embedding. 
It anneals the qubits used, given an anneal schedule (modify the anneal schedule as needed),
and a list of flux bias offsets.
Outputs mi, the magnetization of qubit i, averaged over all reads.
This function is called through the flux bias offset calibration's iterations.

In [28]:
def qub_mags(hifluxl,Nreads,sstar):
    hdict={}
    hbias=[0]*sampler.properties['num_qubits']
    for j in range(len(hifluxl)):
        hbias[int(newnodes[j])]=hifluxl[j]
        hdict.update({newnodes[j]:0})
    
    ann_sch=[[0.0,0.0],[(100*sstar),sstar],[(100.0*sstar)+10.0,sstar],[(100.0*sstar)+10.0+1.0-sstar,1.0]]
        
    resp = sampler.sample_ising(
        h=hdict,J={},num_reads=Nreads,
        flux_drift_compensation=False,
        flux_biases=hbias,
        anneal_schedule=ann_sch
    )
             
    #number of runs in resp
    Nruns=resp.record.shape[0]
    #make a list of qubits used
    qubitl=[]
    for qubit in resp.variables:
        qubitl.append(qubit)
    #calculate the magnetization of each qubit over all runs
    #and make a magnetization list
    magl=[]
    for i in range(len(qubitl)):
        #list of spins from all runs for qubiti
        spinl=[]
        for run in range(Nruns):
            spin=resp.record[run][0][i]
            spinl.append(spin)
        avgspin=(sum(spinl))/(len(spinl))
        magl.append(avgspin)
        
    qm_ar=np.array([qubitl,magl])
    #this is the magnetization, averaged first over all runs for individual sites,
    #and then averaged over all sites
    avg_m=np.mean(qm_ar[1])
    stdev_m=np.std(qm_ar[1])

    
#     dwave.inspector.show(resp)
    
    return qm_ar,avg_m,stdev_m

In [2]:
a=float(5e-6)

5e-06

In [29]:
def binsearch(hiup_l,hidown_l,Nreads,Nrep,sstar):
    print("avg of inputted hiup is", np.mean(hiup_l))
    print("avg of inputted hidown is", np.mean(hidown_l))

    
    check1=True
    count=0
    while check1:
        q_mags=qub_mags(hiup_l,Nreads,sstar)
        if q_mags[1]>0.5:
            check1=False
        hiup_l=hiup_l*2
        print(count,"avg mag=",q_mags[1])
        count+=1
    print("done setting hiups")
    print("avg hiup is", np.mean(hiup_l))
    print("")
    
    check2=True
    count=0
    while check2:
        q_mags=qub_mags(hidown_l,Nreads,sstar)
        if q_mags[1]<-0.5:
            check2=False
        hidown_l=hidown_l*2
        print(count,"avg mag=",q_mags[1])
        count+=1
    print("done setting hidowns")
    print("avg hidown is", np.mean(hidown_l))
    print("")
    
    
    Nqu=len(q_mags[0][1])
    hifluxl=(1/2)*(hiup_l+hidown_l)
    print("initial avg of hiflux is", np.mean(hifluxl))
    print("")
    
    for rep in range(Nrep):
        pivotl=(1/2)*(hiup_l+hidown_l)
        print("current avg pivot is", np.mean(pivotl))
        q_mags=qub_mags(pivotl,Nreads,sstar)

        for i in range(Nqu):
            if q_mags[0][1][i]>0:
                hiup_l[i]=pivotl[i]
            if q_mags[0][1][i]<0:
                hidown_l[i]=pivotl[i]
        print("done with rep",rep)
        print("current avg mag", q_mags[1])
        print("current mag stdev", np.std(q_mags[0][1]))
        print("")
                
    hifluxl=(1/2)*(hiup_l+hidown_l)
    print("avg hiflux is",np.mean(hifluxl))
    variance=np.abs(hiup_l-hidown_l)
    
    return hifluxl,variance

In [30]:
# phimax=sampler.properties['anneal_offset_step_phi0']
# hiuptest=0.1*phimax*np.ones(len(nodes))
# hidowntest=-0.1*phimax*np.ones(len(nodes))
# test=binsearch(hiuptest,hidowntest,1.0)
# #hflx is the list of hiflux values
# hflx=test[0]
# hflxmean=np.mean(hflx[0])
# print(hflxmean)
# hflxstdev=np.std(hflx[0])
# print(hflxstdev)

# h={}
# J=couplers
# hbias=[0]*sampler.properties['num_qubits']
# for j in range(len(hflx)):
#     hbias[int(nodes[j])]=hflx[j]
# response = sampler.sample_ising(h, J, num_reads=100,flux_drift_compensation=False,
#                                flux_biases=hbias)

# print(response)
# dwave.inspector.show(response)

# New flux calibration functions

This new flux calibration routine is based on "Coherent quantum annealing in 2000 qubit chain" - King et al

The same qub mags function is used for the new flux calibration routine

In [44]:
#Nrep is repetitions of calibration
#Nreads is anneals per repetition
def new_flux_cal(mi_init,hifluxl_init,alpha,Nrep,Nreads,sstar):
    
    #intialize the lists of mi and hiflux
    #for the first run this will be 
    mi=mi_init
    hifluxl=hifluxl_init
    
    flux_list_Nrep=[]
    mi_list_Nrep=[]
    
    for count in range(Nrep):
        q_mags=qub_mags(hifluxl,Nreads,sstar)
        mi=q_mags[0][1]
        
        flux_list_Nrep.append(hifluxl)
        mi_list_Nrep.append(mi)
        
        hifluxl=hifluxl-alpha*mi

    return flux_list_Nrep,mi_list_Nrep

# Document name

In [26]:
def DocName(jvalue_internal,Num):
    str1=str(Num)
    
    str2="J="+str(jvalue_internal)
    
    now = datetime.now()
    str3 = now.strftime("%m-%d-%y__%H-%M-%S")
    
    Documentname=str1+"x"+str1+"Kagome"+str2+"__"+str3+".csv"
    
    return Documentname

# Annealing with new flux calibration

qpu annealing

In [42]:
#setting N and J values (globals)
Num=7
j2=-2.0
j3=-3.0
jint=-1.0
jtil=-1.0

#get the registries and all sites qubits array for the whole NxN lattice
registries=NxN_tile(Num)
all_sites_qubits=tile_sites_qubits(Num)


#this block lets us identify missing qubits and stores them
print("these are the base couplers that we want to embed")
couplers=getcouplings(
    registries[0],registries[1],registries[2],registries[3],
j2,j3,jint,jtil)
#extracting tuples from coupler dictionary
tuples_only=[]
for edge in couplers.keys():
    tuples_only.append(edge)
#extracting nodes being used from the couplers
nodes=[]
for i in tuples_only:
    val0=i[0]
    val1=i[1]
    nodes.append(val0)
    nodes.append(val1)
nodes=[*set(nodes)]
nodes.sort()
print("")
print("there are", len(nodes), "nodes")
print("these are the missing qubits")
missingqubits=check_nodes(nodes)
print("")


#get the same couplers again to see which couplers we need to remove
print("these are the base couplers that we want to embed, again")
oldcouplers=getcouplings(
    registries[0],registries[1],registries[2],registries[3],
j2,j3,jint,jtil)
print("")

oldtuples=[]
oldsets=[]
for tupl in oldcouplers.keys():
    oldtuples.append(tupl)
    oldsets.append(set(tupl))

oldinttil=split_oldcouplers(oldcouplers,oldtuples)[0]
old2chains=split_oldcouplers(oldcouplers,oldtuples)[1]
old3chains=split_oldcouplers(oldcouplers,oldtuples)[2]
result_pair=split_oldcouplers(oldcouplers,oldtuples)[3]

print("these are the couplers we want to remove")
removelist=to_remove(missingqubits,oldinttil,old2chains,old3chains,result_pair)
print(removelist)
print("")

#remove the appropriate couplers before calibration
#copy couplers first
newbasecouplers=oldcouplers
newcouplers=remove_couplers(newbasecouplers,oldtuples,removelist)
newtuples=[]
for edge in newcouplers.keys():
    newtuples.append(edge)
print(len(newtuples),"tuples")
newnodes=[]
for tupl in newtuples:
    val0=tupl[0]
    val1=tupl[1]
    newnodes.append(val0)
    newnodes.append(val1)
newnodes=[*set(newnodes)]
newnodes.sort()
print("there are",len(newnodes),"nodes now")
print("")
print("")


print("running new flux calibration")
#flux calibration
#set Nrep for the stages
Nrep1=100
Nrep2=300
Nrep3=400
Nrep_total=Nrep1+Nrep2+Nrep3

#make a list of all hiflux lists from all iterations
all_hiflux=[]
#make a list of all mi lists from all iterations
all_mi=[]

#start by setting all flux bias offsets to 0, and get mi (to be used in very first iteration)
zerofluxes=np.zeros(len(newnodes))
all_hiflux.append(zerofluxes)
mi_zero=qub_mags(zerofluxes)[0][1]
all_mi.append(mi_zero)


#first stage has 0 flux bias offsets (alpha=0)
print("doing",Nrep1,"iterations of alpha=0")
first_run=new_flux_cal(mi_zero,zerofluxes,alpha=0,Nrep=Nrep1,Nreads=100,sstar=0.2)
first_flux_out=first_run[0]
first_mi_out=first_run[1]
#extract the final list of mi and hiflux to be used in the next stage
first_final_flux=first_flux_out[len(first_flux_out)-1]
first_final_mi=first_mi_out[len(first_mi_out)-1]

all_hiflux.append(first_flux_out)
all_mi.append(first_mi_out)
print("done 1st stage")
print("")


#second stage has nonzero alpha
print("doing",Nrep2,"iterations of alpha=5e-6")
second_run=new_flux_cal(first_final_mi,first_final_flux,alpha=5e-6,Nrep=Nrep2,Nreads=100,sstar=0.2)
second_flux_out=second_run[0]
second_mi_out=second_run[1]
#extract the final list of mi and hiflux to be used in the next stage
second_final_flux=second_flux_out[len(second_flux_out)-1]
second_final_mi=second_mi_out[len(second_mi_out)-1]

all_hiflux.append(second_flux_out)
all_mi.append(second_mi_out)
print("done 2nd stage")
print("")


#third stage also has nonzero alpha
print("doing",Nrep2,"iterations of alpha=0.2")
third_run=new_flux_cal(second_final_mi,second_final_flux,alpha=0.2,Nrep=Nrep3,Nreads=100,sstar=0.2)
third_flux_out=third_run[0]
third_mi_out=third_run[1]

all_hiflux.append(third_flux_out)
all_mi.append(third_mi_out)
print("done 3rd stage")
print("")

#checking the big lists' shape
print("checking if we have the correct number of values")
print("there are", len(all_hiflux), "stages in the hiflux list")
print("there are", len(all_mi), "stages in the mi list")
print("")
print(
    "there are," len(all_hiflux[0]), len(all_hiflux[1]), len(all_hiflux[2]),
                                                             "hiflux lists in stage 1, 2, and 3")
print(
    "there are," len(all_mi[0]), len(all_mi[1]), len(all_mi[2]),
                                                             "mi lists in stage 1, 2, and 3")

#all_hiflux should help us make the hairy plots in King et al

these are the base couplers that we want to embed
There are 196 2chains
There are 784 3chains (when split into 2chains)
There are 833 internal couplings
There are 126 x tiling couplings
There are 126 y tiling couplings
There are 36 xy couplings
There are 2101 total couplings

there are 1568 nodes
these are the missing qubits
46.0 is not found in list
524.0 is not found in list
548.0 is not found in list
1723.0 is not found in list
1735.0 is not found in list

these are the base couplers that we want to embed, again
There are 196 2chains
There are 784 3chains (when split into 2chains)
There are 833 internal couplings
There are 126 x tiling couplings
There are 126 y tiling couplings
There are 36 xy couplings
There are 2101 total couplings

these are the couplers we want to remove
[(34.0, 38.0), (46.0, 38.0), (524.0, 532.0), (521.0, 524.0), (540.0, 548.0), (537.0, 540.0), (1723.0, 1727.0), (1851.0, 1723.0), (1730.0, 1735.0), (521.0, 525.0), (409.0, 537.0), (393.0, 521.0), (1719.0, 1727.0)